Optimizers

Task 1.

In [ ]:
import time
import numpy as np
from .optimization.optimizators import bfgs, lbfgs, adam

def test_function(x: np.ndarray) -> float:
    assert len(x) == 3

    term1 = 0.5 * (x[0] ** 2)
    term2 = (x[0] - x[1]) ** 2 + (x[1] - x[2]) ** 2
    term3 = x[2] ** 2
    term4 = -x[0]

    return term1 + term2 + term3 + term4

x0 = np.array([1.0, 1.0, 1.0])

def test_bfgs():
    start_time = time.time()
    x_opt_bfgs = bfgs(test_function, x0)
    duration_bfgs = time.time() - start_time
    return x_opt_bfgs, duration_bfgs

def test_lbfgs():
    start_time = time.time()
    x_opt_lbfgs = lbfgs(test_function, x0)
    duration_lbfgs = time.time() - start_time
    return x_opt_lbfgs, duration_lbfgs

def test_adam():
    start_time = time.time()
    x_opt_adam = adam(test_function, x0)
    duration_adam = time.time() - start_time
    return x_opt_adam, duration_adam

x_opt_bfgs, duration_bfgs = test_bfgs()
x_opt_lbfgs, duration_lbfgs = test_lbfgs()
x_opt_adam, duration_adam = test_adam()

print("Results:")
print(f"BFGS: Optimal x = {x_opt_bfgs}, Time taken: {duration_bfgs:.5f} seconds")
print(f"L-BFGS: Optimal x = {x_opt_lbfgs}, Time taken: {duration_lbfgs:.5f} seconds")
print(f"Adam: Optimal x = {x_opt_adam}, Time taken: {duration_adam:.5f} seconds")


Task 2

In [14]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

class ThreeLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ThreeLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        hidden_output = self.relu(self.fc1(X))
        output = self.sigmoid(self.fc2(hidden_output))
        return output

    def get_parameters(self):
        params = []
        for param in self.parameters():
            params.append(param.detach().numpy().flatten())
        return np.concatenate(params)

    def set_parameters(self, params):
        param_idx = 0
        with torch.no_grad():
            for param in self.parameters():
                param_size = param.numel()
                new_values = torch.tensor(params[param_idx:param_idx + param_size]).reshape(param.size())
                param.copy_(new_values)
                param_idx += param_size

    def loss(self, X, y):
        output = self.forward(X)
        criterion = nn.BCELoss()
        return criterion(output, y).item()

def generate_data(n_samples=200):
    X = torch.randn(n_samples, 2)
    y = (X[:, 0] + X[:, 1] > 0).float().reshape(-1, 1)
    return X, y

def train_custom_optimizer(model, X, y, optimizer_func, grad_method='forward_diff', num_epochs=100, h=1e-5):
    X_np = X.detach().numpy()
    y_np = y.detach().numpy()

    losses = []
    for epoch in range(num_epochs):
        def model_loss(params):
            model.set_parameters(params)
            return model.loss(torch.tensor(X_np).float(), torch.tensor(y_np).float())

        params = model.get_parameters()

        params = optimizer_func(model_loss, params, grad_method=grad_method, h=h, max_iter=1)

        model.set_parameters(params)

        loss_value = model_loss(params)
        losses.append(loss_value)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_value:.4f}')

    return losses

def visualize_loss(loss_bfgs, loss_lbfgs, loss_adam):
    plt.figure(figsize=(10, 6))
    plt.plot(loss_bfgs, label="BFGS", color="blue")
    plt.plot(loss_lbfgs, label="L-BFGS", color="green")
    plt.plot(loss_adam, label="Adam", color="red")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epochs for BFGS, L-BFGS, Adam')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
input_size = 2
hidden_size = 30
num_epochs = 100

X, y = generate_data(n_samples=200)

model_bfgs = ThreeLayerPerceptron(input_size=input_size, hidden_size=hidden_size)
model_lbfgs = ThreeLayerPerceptron(input_size=input_size, hidden_size=hidden_size)
model_adam = ThreeLayerPerceptron(input_size=input_size, hidden_size=hidden_size)

print("Training with BFGS optimizer...")
loss_bfgs = train_custom_optimizer(model_bfgs, X, y, bfgs, grad_method='forward_diff', num_epochs=num_epochs)

print("\nTraining with L-BFGS optimizer...")
loss_lbfgs = train_custom_optimizer(model_lbfgs, X, y, lbfgs, grad_method='forward_diff', num_epochs=num_epochs)

print("\nTraining with Adam optimizer...")
loss_adam = train_custom_optimizer(model_adam, X, y, adam, grad_method='forward_diff', num_epochs=num_epochs)

visualize_loss(loss_bfgs, loss_lbfgs, loss_adam)